In [1]:
import pandas as pd

data = pd.read_csv("../data/design-seeds.csv")

In [2]:
multiplier = 24

In [3]:
from repalette.utils.data import PairRecolorDataset

In [4]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size=0.2)
val, test = train_test_split(test, test_size=0.5)

In [5]:
from repalette.utils.data import ShuffleDataLoader

train_dataset = PairRecolorDataset(train, multiplier)
train_dataloader = ShuffleDataLoader(train_dataset, shuffle=False, num_workers=8, batch_size=8)

In [6]:
len(train_dataset)

38640

In [7]:
val_dataset = PairRecolorDataset(val, multiplier)
val_dataloader = ShuffleDataLoader(val_dataset, shuffle=False, num_workers=8, batch_size=8)

In [8]:
len(val_dataset)

4968

In [9]:
test_dataset = PairRecolorDataset(test, multiplier)
test_dataloader = ShuffleDataLoader(test_dataset, shuffle=False, num_workers=8, batch_size=8)

In [10]:
from repalette.models import PaletteNet

In [11]:
import torch
import torch.nn as nn
import pytorch_lightning as pl

In [12]:
from repalette.models import PaletteNet

In [13]:
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger
import os

from repalette.constants import DEFAULT_LR, DEFAULT_BETAS, PL_LOGS_DIR, MODELS_DIR

if not os.path.exists(MODELS_DIR):
    os.mkdir(MODELS_DIR)
    
if not os.path.exists(PL_LOGS_DIR):
    os.mkdir(PL_LOGS_DIR)

checkpoint_callback = ModelCheckpoint(
    filepath=MODELS_DIR,
    monitor="Val/Loss",
    verbose=True,
    mode='min',
    save_top_k=2
)

early_stop_callback = EarlyStopping(
   monitor="Val/Loss",
   min_delta=0.00,
   patience=20,
   verbose=False,
   mode='min'
)

hparams={
    'lr': DEFAULT_LR/5, 'betas': DEFAULT_BETAS,
}
model = PaletteNet(
    train_dataloader, val_dataloader, test_dataloader,
    hparams=hparams,
)
logger = TensorBoardLogger(PL_LOGS_DIR, name='PaletteNet')
trainer = Trainer(
    gpus=1, # use gpu!
    logger=logger,
    callbacks=[early_stop_callback],
    checkpoint_callback=checkpoint_callback,
#     auto_lr_find="learning_rate"
)

# trainer.tune(model)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


In [14]:
# train_iter = iter(model.train_dataloader())

In [15]:
# batch = next(train_iter)

In [16]:
trainer.fit(model)


  | Name               | Type              | Params
---------------------------------------------------------
0 | feature_extractor  | FeatureExtractor  | 11 M  
1 | recoloring_decoder | RecoloringDecoder | 2 M   
2 | loss_fn            | MSELoss           | 0     


FileNotFoundError: Caught FileNotFoundError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/dan/miniconda3/envs/repalette/lib/python3.8/site-packages/torch/utils/data/_utils/worker.py", line 185, in _worker_loop
    data = fetcher.fetch(index)
  File "/home/dan/miniconda3/envs/repalette/lib/python3.8/site-packages/torch/utils/data/_utils/fetch.py", line 44, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/dan/miniconda3/envs/repalette/lib/python3.8/site-packages/torch/utils/data/_utils/fetch.py", line 44, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/dan/Projects/repalette/repalette/utils/data.py", line 111, in __getitem__
    image = Image.open(self.path_prefix + str(self.data["image_path"].iloc[i]))
  File "/home/dan/miniconda3/envs/repalette/lib/python3.8/site-packages/PIL/Image.py", line 2878, in open
    fp = builtins.open(filename, "rb")
FileNotFoundErro